In [ ]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from pathlib import Path

In [ ]:
p = Path("../").resolve()

In [ ]:
import json
import os
from collections import Counter
from io import BytesIO
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union, cast
import cv2
import matplotlib as plt
import numpy as np
import pandas as pd
import PIL
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from tqdm import tqdm

from geoscreens.consts import GEO_SCREENS, IMG_SIZE
from geoscreens.geo_data import GeoScreensDataModule
from geoscreens.modules import LightModelTorch, build_module
from scripts.train_geo import get_model

USE_RAY = False
if USE_RAY:
    import ray

    ray.init(ignore_reinit_error=True)

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
plt.rcParams["figure.figsize"] = (12, 10)

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

In [ ]:
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("pdf", "png")
plt.rcParams["savefig.dpi"] = 75

plt.rcParams["figure.autolayout"] = False
plt.rcParams["figure.figsize"] = 10, 6
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 2.0
plt.rcParams["lines.markersize"] = 8
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["text.usetex"] = True

plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "cm"
# plt.rcParams["text.latex.preamble"] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

In [ ]:
tasks = json.load(open(Path("/shared/gbiamby/geo/geoscreens_003_tasks_export.json")))
print(len(tasks), " total tasks")
tasks[:2]

In [ ]:
for t in tqdm(tasks):
    t["data"]["full_path"] = t["data"]["image"].replace(
        "/data/local-files/?d=", "/shared/gbiamby/geo/screenshots/"
    )
    width, height = PIL.Image.open(t["data"]["full_path"]).size
    t["data"]["width"] = width
    t["data"]["height"] = height

In [ ]:
Counter([(img["data"]["width"], img["data"]["height"] ) for img in tasks])

In [ ]:
img_sizes = {t["data"]["full_path"]: t["data"] for t in tasks}
list(img_sizes.items())[:10]

In [ ]:
import pickle
# list(img_sizes.items())[:10]
pickle.dump(img_sizes, open("./img_sizes.pkl", "wb"))

In [ ]:
tasks[0]

In [ ]:
mistakes = []
for i, t in enumerate(tqdm(tasks, total=len(tasks))):
    # if i >= 10:
    #     break
    # print("")
    anns_results = [ann["result"] for ann in t["annotations"]]
    # print(anns_results)
    # print([ann for ann in anns_results])
    # if ann["value"] and ann["value"]["rectanglelables"]:
    if anns_results:
        labels = [ann["value"]["rectanglelabels"][0] for ann in anns_results[0]]
        if len(labels) != len(set(labels)):
            mistakes.append(t)

In [ ]:
len(mistakes)

In [ ]:
[m["data"] for m in mistakes]